In [ ]:
!pip install requests pandas

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [16]:
http://apis.data.go.kr/B551011/GoCamping/basedList?serviceKey=ServiceKey&numOfRows=10&pageNo=1&MobileOS=ETC&MobileApp=TestApp&_type=json



SyntaxError: invalid syntax (276766137.py, line 1)

In [17]:
http://apis.data.go.kr/B551011/GoCamping/locationBasedList?serviceKey=서비스인증키&MobileOS=ETC&MobileApp=AppTest&mapX=128.6142847&mapY=36.0345423&radius=2000

SyntaxError: invalid syntax (2468142551.py, line 1)

In [ ]:
import os
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

response = openai.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {
      "role": "system",
      "content": "다음의 주어진 설명에 맞는 한글 제품명을 10개 생성하세요."
    },
    {
      "role": "user",
      "content": """
제품: 가정용 슬러시 제조기
특징: 빠르다, 싸다, 관리가 편하다
"""
    }
  ],
  temperature=0.8,
  max_tokens=256,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

print(response.choices[0].message.content)

In [15]:



import requests
import pandas as pd
from urllib.parse import urlencode

# ✅ 반드시 "디코딩된" 인증키 사용!
API_KEY = "jSEJtgH1oBCQkxYx3vrdl3NTc1X4YVOZRSgk2lh83kzuLXV0ONZJEpjC/da0J/85gbZ2fgG1/u1iMd8v0LkZqQ=="

# ✅ JSON 지원하는 API로 변경!
BASE_URL = "http://apis.data.go.kr/B551011/GoCamping/searchList"

def search_camping_sites(keyword=None, region=None):
    params = {
        'MobileOS': 'ETC',
        'MobileApp': 'campingApp',
        'numOfRows': 10,
        'pageNo': 1,
        'format': 'json'
    }

    if keyword:
        params['keyword'] = keyword
    if region:
        params['doNm'] = region

    query_string = urlencode(params, safe='=&')
    # ✅ 인증키는 맨 끝에 직접 붙이기
    url = f"{BASE_URL}?{query_string}&serviceKey={API_KEY}"
    print(f"[DEBUG] 요청 URL: {url}")

    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()

        content_type = response.headers.get('Content-Type', '')
        print(f"[DEBUG] 응답 Content-Type: {content_type}")

        if 'json' not in content_type:
            print("❌ JSON 응답이 아닙니다.")
            print(response.text[:500])
            return []

        data = response.json()

        if 'response' not in data or 'body' not in data['response']:
            print("⚠️ 응답 구조가 예상과 다릅니다.")
            return []

        return data['response']['body']['items']['item']

    except Exception as e:
        print(f"❌ 오류 발생: {e}")
        return []

def recommend_camping_site(user_input):
    print(f"\n[사용자 입력] {user_input}\n")

    keyword = None
    region = None

    if "강원" in user_input:
        region = "강원도"
    if "계곡" in user_input:
        keyword = "계곡"
    elif "해변" in user_input or "바다" in user_input:
        keyword = "해변"
    elif "숲" in user_input:
        keyword = "숲"
    elif not keyword:
        keyword = user_input

    sites = search_camping_sites(keyword=keyword, region=region)

    if not sites:
        print("😢 조건에 맞는 캠핑장을 찾을 수 없습니다.")
        return

    df = pd.DataFrame(sites)
    print("\n🎯 추천 캠핑장 상위 5개:")
    print(df[['facltNm', 'lineIntro', 'addr1', 'tel', 'homepage']].head())

if __name__ == "__main__":
    user_question = "강원도 계곡 근처 캠핑장 추천"
    recommend_camping_site(user_question)


[사용자 입력] 강원도 계곡 근처 캠핑장 추천

[DEBUG] 요청 URL: http://apis.data.go.kr/B551011/GoCamping/searchList?MobileOS=ETC&MobileApp=campingApp&numOfRows=10&pageNo=1&format=json&keyword=%EA%B3%84%EA%B3%A1&doNm=%EA%B0%95%EC%9B%90%EB%8F%84&serviceKey=jSEJtgH1oBCQkxYx3vrdl3NTc1X4YVOZRSgk2lh83kzuLXV0ONZJEpjC/da0J/85gbZ2fgG1/u1iMd8v0LkZqQ==
[DEBUG] 응답 Content-Type: application/xml
❌ JSON 응답이 아닙니다.
<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><header>
  <responseTime>2025-07-21T15:48:01.084</responseTime>
  <resultCode>10</resultCode>
  <resultMsg>INVALID_REQUEST_PARAMETER_ERROR(format)</resultMsg>
</header></response>
😢 조건에 맞는 캠핑장을 찾을 수 없습니다.
